## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-08-08-10-52-48 +0000,nyt,Israel Says It’s Preparing to Take Control of ...,https://www.nytimes.com/2025/08/08/world/middl...
1,2025-08-08-10-52-38 +0000,nypost,Historic New Jersey synagogue engulfed by flam...,https://nypost.com/2025/08/08/us-news/new-jers...
2,2025-08-08-10-51-05 +0000,bbc,Ex-president can be buried in Zambia against f...,https://www.bbc.com/news/articles/c4gz2dg9wzwo...
3,2025-08-08-10-46-00 +0000,wsj,Stock Market Today: Gold Prices Hit Record Hig...,https://www.wsj.com/livecoverage/stock-market-...
4,2025-08-08-10-42-24 +0000,nyt,"As Gaza Hunger Crisis Worsens, Israel Lets in ...",https://www.nytimes.com/2025/08/08/world/middl...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2310/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
49,trump,78
3,gaza,21
7,new,18
343,tariffs,17
214,california,12


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
165,2025-08-07-21-50-00 +0000,wsj,President Trump said he would nominate Stephen...,https://www.wsj.com/politics/policy/stephen-mi...,130
215,2025-08-07-20-10-22 +0000,wapo,Trump celebrates new tariffs as negotiations c...,https://www.washingtonpost.com/business/2025/0...,126
334,2025-08-07-11-54-12 +0000,bbc,Trump's sweeping new tariffs take effect again...,https://www.bbc.com/news/articles/cx23jmvn5yzo...,125
303,2025-08-07-14-30-28 +0000,wapo,Trump’s new tariffs are hitting these major tr...,https://www.washingtonpost.com/business/2025/0...,125
232,2025-08-07-19-02-30 +0000,wapo,Here’s where the money from Trump’s tariffs wi...,https://www.washingtonpost.com/business/2025/0...,122


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
165,130,2025-08-07-21-50-00 +0000,wsj,President Trump said he would nominate Stephen...,https://www.wsj.com/politics/policy/stephen-mi...
35,76,2025-08-08-09-35-51 +0000,bbc,Israel approves plan to take control of Gaza C...,https://www.bbc.com/news/articles/c8ryekj1m1do...
234,61,2025-08-07-18-55-16 +0000,nypost,New NYC Council proposal will tie up businesse...,https://nypost.com/2025/08/07/us-news/new-nyc-...
124,47,2025-08-08-00-24-53 +0000,bbc,"Dozens of migrants detained under 'one in, one...",https://www.bbc.com/news/articles/ce35v0zyzvlo...
311,39,2025-08-07-13-55-44 +0000,nypost,Trump demands Intel CEO Lip-Bu Tan ‘resign imm...,https://nypost.com/2025/08/07/business/trump-d...
146,39,2025-08-07-22-40-47 +0000,nypost,Disney settles Gina Carano’s lawsuit after fir...,https://nypost.com/2025/08/07/media/disney-set...
213,38,2025-08-07-20-17-49 +0000,latimes,California Supreme Court sides with environmen...,https://www.latimes.com/environment/story/2025...
257,36,2025-08-07-17-51-00 +0000,wsj,President Trump’s tariff rollout has taken num...,https://www.wsj.com/economy/trade/trump-tariff...
242,35,2025-08-07-18-30-00 +0000,wsj,Sen. John Cornyn said the FBI will help track ...,https://www.wsj.com/politics/policy/fbi-texas-...
202,34,2025-08-07-20-52-50 +0000,nyt,Staggering U.S. Tariffs Begin as Trump Widens ...,https://www.nytimes.com/2025/08/07/business/ec...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
